In [ ]:
# !pip install quandl
# import quandl

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import get_data_yahoo as gd_y
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from matplotlib import pyplot

In [ ]:
start = dt.datetime(2005, 1, 1)
end = dt.datetime(2020, 8, 21)
# end = dt.date.today()
df = gd_y('^NSEI',start,end)["Adj Close"]
df = pd.DataFrame(df)

seq_length = 1000

In [ ]:
x = np.array(df['Adj Close'])
y = np.array(df.index)
df1 = pd.DataFrame(x, columns=['Close'])
df2=pd.DataFrame(y, columns=['Date'])
df = pd.concat([df1,df2], axis=1)
df.set_index('Date')

In [ ]:
#creating dataframe
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]


In [ ]:
#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

In [ ]:
#creating train and test sets
dataset = new_data.values


train = dataset[0:(dataset.shape[0] - seq_length),:]
valid = dataset[(dataset.shape[0] - seq_length):,:]

In [ ]:
valid.shape

In [ ]:
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
scaled_data

In [ ]:
x_train, y_train = [], []
for i in range(seq_length,len(train)):
    x_train.append(scaled_data[i-seq_length:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(x_train, y_train, validation_split=.2, epochs=3, batch_size=16, verbose=1)

## Prediction on Current *Data*

In [ ]:
#predicting 1 value, using past seq_length from the train data
inputs = new_data[len(new_data) - len(valid) - seq_length:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)


In [ ]:
X_test = []
for i in range(seq_length,inputs.shape[0]):
    X_test.append(inputs[i-seq_length:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))


In [ ]:
closing_price = model.predict(X_test)

In [ ]:
closing_price = scaler.inverse_transform(closing_price)

# Tomorrow's Closing Price

In [ ]:
# closing_price[-1]
# closing_price
validation = pd.DataFrame(closing_price, columns=["Prediciton"])
validation['Actual'] = np.array(df['Close'][-1001:])
validation

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))


## **Plot**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#for plotting
train = new_data[:(dataset.shape[0] - seq_length)]
valid = new_data[(dataset.shape[0] - seq_length):]
valid['Predictions'] = closing_price

fig, ax = plt.subplots(figsize=(20,10))
ax.plot(train['Close'], label="Train")
ax.plot(valid['Close'], label="Actual")
ax.plot(valid['Predictions'], label="Predictions")
leg = ax.legend()

## **Training on Log differences to Predict Change in Price**

In [ ]:
df1 = pd.DataFrame(df)
df1['Log']=np.log(df1['Close'])
df1['Log_diff'] = df1['Log'].diff(1) *100
df1= df1.drop(columns= ['Close', 'Log'])
df1 =  df1.dropna()
df1

In [ ]:
#setting index
new_data = pd.DataFrame(df1)
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

In [ ]:
#creating train and test sets
dataset = new_data.values

train = dataset[0:(dataset.shape[0] - seq_length),:]
valid = dataset[(dataset.shape[0] - seq_length):,:]

In [ ]:
dataset

In [ ]:
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)
# scaled_data = dataset
scaled_data = np.asarray(scaled_data).astype(np.float32)
scaled_data

In [ ]:
x_train, y_train = [], []
for i in range(seq_length,len(train)):
    x_train.append(scaled_data[i-seq_length:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(x_train, y_train, validation_split=.2, epochs=1, batch_size=16, verbose=1)

**PREDICTING ON CURRENT DATA**

In [ ]:
#predicting 1 value, using past seq_length from the train data
inputs = new_data[len(new_data) - len(valid) - seq_length:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

In [ ]:
X_test = []
for i in range(seq_length,inputs.shape[0]):
    X_test.append(inputs[i-seq_length:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))


In [ ]:
Log_diff = model.predict(X_test)

In [ ]:
Log_diff

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#for plotting
train = new_data[:(dataset.shape[0] - seq_length)]
valid = new_data[(dataset.shape[0] - seq_length):]
valid['Predictions'] = Log_diff

fig, ax = plt.subplots(figsize=(20,10))
ax.plot(train['Log_diff'], label="Train")
ax.plot(valid['Log_diff'], label="Actual")
ax.plot(valid['Predictions'], label="Predictions")
leg = ax.legend()